In [7]:
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import datetime
import time
from multiprocessing.dummy import Pool

In [8]:
df = pd.read_csv('main_task.zip')
df2 = pd.read_csv('kaggle_task.zip')
df = df.append(df2, sort=False).reset_index(drop=True)[['URL_TA']]
df['URL_TA_ind2'] = df['URL_TA'].apply(lambda x: x.split("-")[2])
df['parser_status'] = -9

df['city_R1'] = ""
df['city_R2'] = ""

df['R_Food'] = 0
df['R_Service'] = 0
df['R_Value'] = 0
df['R_Atmosphere'] = 0

df['R5_Excellent'] = 0
df['R4_Very_good'] = 0
df['R3_Average'] = 0
df['R2_Poor'] = 0
df['R1_Terrible'] = 0

HOST = 'https://www.tripadvisor.com'

In [14]:
def get_page(url, params = None, doc_type = 'html.parser', time_sleep = 0): #'html.parser'  'lxml'
    try:
        res = requests.get(url, headers={'User-Agent': UserAgent().chrome}, params = params)
        time.sleep(time_sleep) 
        if res.status_code == 200:
            page = BeautifulSoup(res.content, doc_type)
            return page
        else:
            return None
    except:
        return None

def page_parser(page):
    save_data = dict()
    try:
        data = page.find_all(class_='_3-W4EexF')
        if data is not None:
            for i in range(len(data)):
                save_data['city_R' + str(i+1)] = data[i].text
                if i == 1:
                    break
    except:
        pass
    
    try:
        data = page.find_all(class_='jT_QMHn2')
        pattern = re.compile("ui_bubble_rating bubble_(\d*)")
        if data is not None:
            for i in range(len(data)):
                if data[i].text in ['Food','Service','Value','Atmosphere']:
                    save_data['R_' + data[i].text] = pattern.findall(str(data[i].find(class_='_377onWB-')))[0]
    except:
        pass
    
    try:
        data = page.find_all(class_="row_num is-shown-at-tablet")
        if data is not None:
            if len(data) == 5:
                save_data['R5_Excellent'] = data[0].text
                save_data['R4_Very_good'] = data[1].text
                save_data['R3_Average'] = data[2].text
                save_data['R2_Poor'] = data[3].text
                save_data['R1_Terrible'] = data[4].text
    except:
        pass
            
    return save_data

def iter_parser(df_index_range = []):
    if df_index_range != []:
        if isinstance(df_index_range, int):
            df_index_range = [df_index_range]
            
        for index_ in df_index_range:
            page = get_page(HOST + df['URL_TA'][index_])
            if page is None:
                df.loc[index_, 'parser_status'] = -1
                continue
            data_ = page_parser(page)
            if data_ == {}:
                df.loc[index_, 'parser_status'] = 0
                continue
            
            df.loc[index_, 'parser_status'] = 1
            try:
                df.update(pd.DataFrame(data_, index = [index_]))
            except:
                df.loc[index_, 'parser_status'] = -2         
            
def pool_parser(index_ = 0):
    page = get_page(HOST + df['URL_TA'][index_])
    if page is None:
        df.loc[index_, 'parser_status'] = -1
        return 

    data_ = page_parser(page)
    if data_ == {}:
        df.loc[index_, 'parser_status'] = 0
        return 

    df.loc[index_, 'parser_status'] = 1
    try:        
        df.update(pd.DataFrame(data_, index = [index_]))
    except:
        df.loc[index_, 'parser_status'] = -2 

In [10]:
# %%time
# iter_parser(df_index_range = 3) #df.shape[0]
# df.to_csv('dop_feat_parser.zip', index=True)

In [13]:
%%time
part = 1000
for i in range(10, 11):
    start = datetime.datetime.now()
    print(i," - start - ", start, " : ", end = "")
    pool = Pool(20)
    results = pool.map(pool_parser, list(range(part*i, part*(i+1)))) #df.shape[0]
    pool.terminate()
    print((datetime.datetime.now() - start).total_seconds())
    df[part*i: part*(i+1)].to_csv('dop_feat_parser_'+str(i)+'.zip', index=True)

10  - start -  2020-07-27 11:24:40.884559  : 29.877257
Wall time: 29.9 s


In [87]:
df_dop = pd.read_csv('myparser_result\dop_feat_parser_0.zip', index_col = 'Unnamed: 0')
for i in range(1, 50):
    df_part = pd.read_csv('myparser_result\dop_feat_parser_' + str(i) + '.zip', index_col = 'Unnamed: 0')
    df_dop = df_dop.append(df_part, sort=False)

df_dop["city_R1"].fillna("",inplace = True)
df_dop["city_R2"].fillna("",inplace = True)
df_dop.R5_Excellent = df_dop.R5_Excellent.apply(lambda x: str(x).replace(",","")).astype('int64')
df_dop.R4_Very_good = df_dop.R4_Very_good.apply(lambda x: str(x).replace(",","")).astype('int64')
df_dop.R3_Average = df_dop.R3_Average.apply(lambda x: str(x).replace(",","")).astype('int64')
df_dop.R2_Poor = df_dop.R2_Poor.apply(lambda x: str(x).replace(",","")).astype('int64')
df_dop.R1_Terrible = df_dop.R1_Terrible.apply(lambda x: str(x).replace(",","")).astype('int64')
df_dop['parser_status'] = df_dop['parser_status'].apply(lambda x: 0 if x == -1 else x)

df_dop['check'] = df_dop[df_dop.columns[5:]].sum(axis = 1)
df_dop['parser_status'] = df_dop.apply(lambda x: 0 if x['check'] == 0 and x['parser_status'] == -9 
                                       else x['parser_status'], axis = 1)
df_dop['parser_status'] = df_dop.apply(lambda x: 1 if x['check'] != 0 and x['parser_status'] == -9 
                                       else x['parser_status'], axis = 1)
df_dop.drop(columns = ['check'], inplace = True)

df_dop.to_csv('dop_feat_parser_fin.zip', index=True)

df_dop.shape

(50000, 14)

In [85]:
df_dop.isnull().sum()

URL_TA           0
URL_TA_ind2      0
parser_status    0
city_R1          0
city_R2          0
R_Food           0
R_Service        0
R_Value          0
R_Atmosphere     0
R5_Excellent     0
R4_Very_good     0
R3_Average       0
R2_Poor          0
R1_Terrible      0
dtype: int64